In [ ]:
# stock_prediction.py
"""
股票价格预测线性回归模型
功能：
1. 自动数据预处理
2. 特征工程标准化
3. 交叉验证评估
4. 模型训练与预测
5. 结果输出为CSV文件

使用说明：
1. 修改文件路径和列名配置
2. 直接运行脚本即可生成预测结果
"""

# 导入所需库
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import warnings

# 1. 配置参数部分 ==============================================

# 文件路径配置
TRAIN_DATA_PATH = "/kaggle/input/houkong-moai/customer_churn.csv"  # 训练集文件路径
TEST_DATA_PATH = "/kaggle/input/houkong-moai/customer_churn.csv"    # 测试集文件路径

# 数据列配置
TARGET_COLUMN = "churn"  # 要预测的目标列名
DROP_COLUMNS = ["customer_id","gender"]  # 需要删除的无用列
FEATURE_COLUMNS = [      # 使用的特征列
    "age",
    "subscription_length",
    "monthly_bill",
    "total_usage",
    "service_complaints"
]

# 2. 主程序部分 ==============================================

if __name__ == "__main__":
    # 忽略警告信息
    warnings.filterwarnings('ignore')

    # 2.1 数据加载 ==========================================
    print("正在加载数据...")
    try:
        # 读取训练集和测试集
        train_df = pd.read_csv(TRAIN_DATA_PATH)
        test_df = pd.read_csv(TEST_DATA_PATH)
        print(f"数据加载成功！训练集: {train_df.shape}, 测试集: {test_df.shape}")
    except Exception as e:
        print(f"数据加载失败: {str(e)}")
        exit()

    # 2.2 数据预处理 ========================================
    print("\n正在进行数据预处理...")



    # 提取特征和目标值
    X_train = train_df[FEATURE_COLUMNS].values
    y_train = train_df[TARGET_COLUMN].values
    X_test = test_df[FEATURE_COLUMNS].values

    # 2.3 特征工程 ==========================================
    print("\n正在进行特征标准化...")

    # 初始化标准化器
    scaler = StandardScaler()

    # 标准化训练数据
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 2.4 模型训练 ==========================================
    print("\n正在训练模型...")

    # 初始化线性回归模型
    model = LinearRegression()

    # 交叉验证评估
    print("正在进行交叉验证...")
    cv_scores = cross_val_score(
        model,
        X_train_scaled,
        y_train,
        cv=5,
        scoring="neg_mean_squared_error"
    )
    rmse_scores = np.sqrt(-cv_scores)
    print(f"交叉验证结果 - 平均RMSE: {rmse_scores.mean():.4f} (±{rmse_scores.std():.4f})")

    # 训练最终模型
    model.fit(X_train_scaled, y_train)
    print("模型训练完成！")

    # 2.5 预测与评估 ========================================
    print("\n正在进行预测...")

    # 训练集评估
    train_pred = model.predict(X_train_scaled)
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    print(f"训练集RMSE: {train_rmse:.4f}")

    # 测试集预测
    test_pred = model.predict(X_test_scaled)

    # 2.6 结果保存 ==========================================
    print("\n正在保存预测结果...")

    # 创建结果DataFrame
    result_df = pd.DataFrame({
        "ID": test_df["customer_id"],  # 保留日期列
        "prediction": test_pred   # 预测值
    })

    # 保存为CSV文件
    result_df.to_csv("predictions.csv", index=False)
    print("预测结果已保存至 predictions.csv")

    # 2.7 输出重要信息 ======================================
    print("\n===== 模型摘要 =====")
    print(f"使用的特征: {FEATURE_COLUMNS}")
    print(f"删除的列: {DROP_COLUMNS}")
    print(f"训练样本数: {len(X_train)}")
    print(f"测试样本数: {len(X_test)}")
    print("=" * 30)
